In [134]:
import pandas as pd 
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.palettes import Spectral3
from bokeh.transform import factor_cmap

        specifying the output file name     

In [135]:
output_file('4_thor4_resample.html')

        read date       

In [136]:
data = pd.read_csv('thor_wwii.csv')
data.head()

,MSNDATE,THEATER,COUNTRY_FLYING_MISSION,NAF,UNIT_ID,AIRCRAFT_NAME,AC_ATTACKING,TAKEOFF_BASE,TAKEOFF_COUNTRY,TAKEOFF_LATITUDE,TAKEOFF_LONGITUDE,TGT_COUNTRY,TGT_LOCATION,TGT_LATITUDE,TGT_LONGITUDE,TONS_HE,TONS_IC,TONS_FRAG,TOTAL_TONS
0,03/30/1941,ETO,GREAT BRITAIN,RAF,84 SQDN,BLENHEIM,10.0,NaN,NaN,NaN,NaN,ALBANIA,ELBASAN,41.10,20.07,0.0,0.0,0.0,0.0
1,11/24/1940,ETO,GREAT BRITAIN,RAF,211 SQDN,BLENHEIM,9.0,NaN,NaN,NaN,NaN,ALBANIA,DURAZZO,41.32,19.45,0.0,0.0,0.0,0.0
2,12/04/1940,ETO,GREAT BRITAIN,RAF,211 SQDN,BLENHEIM,9.0,NaN,NaN,NaN,NaN,ALBANIA,TEPELENE,40.30,20.02,0.0,0.0,0.0,0.0
3,12/31/1940,ETO,GREAT BRITAIN,RAF,211 SQDN,BLENHEIM,9.0,NaN,NaN,NaN,NaN,ALBANIA,VALONA,40.47,19.49,0.0,0.0,0.0,0.0
4,01/06/1941,ETO,GREAT BRITAIN,RAF,211 SQDN,BLENHEIM,9.0,NaN,NaN,NaN,NaN,ALBANIA,VALONA,40.47,19.49,0.0,0.0,0.0,0.0


        Convert MSNDATE column to pandas (datetime)     

In [137]:
data['MSNDATE'] = pd.to_datetime(data['MSNDATE'], format= '%m/%d/%Y')

        groupby to sum explosion types based on time(resample based Month End)        

In [138]:
dataGrouped = data.groupby(pd.Grouper(key='MSNDATE', freq= 'ME'))[['TOTAL_TONS','TONS_FRAG', 'TONS_IC', 'TONS_HE']].sum()
dataGrouped


,TOTAL_TONS,TONS_FRAG,TONS_IC,TONS_HE
MSNDATE,,,,
1939-09-30,0.0,0.0,0.0,0.0
1939-10-31,0.0,0.0,0.0,0.0
1939-11-30,0.0,0.0,0.0,0.0
1939-12-31,0.0,0.0,0.0,0.0
1940-01-31,0.0,0.0,0.0,0.0
...,...,...,...,...
1945-08-31,61003.1,980.0,14212.0,45824.1
1945-09-30,36.0,0.0,0.0,36.0
1945-10-31,0.0,0.0,0.0,0.0


        convert data to DataSource      

In [139]:
dataSource = ColumnDataSource(dataGrouped)

In [140]:
# countries = dataSource.data['COUNTRY_FLYING_MISSION'].tolist()

        with matplotlib     

In [141]:
 import matplotlib.pyplot as plt

 plt.figure(figsize=(12, 6))
 plt.plot(dataGrouped.index, dataGrouped['TOTAL_TONS'], label='کل انفجارها')
 plt.plot(dataGrouped.index, dataGrouped['TONS_FRAG'], label='قطعات')
 plt.plot(dataGrouped.index, dataGrouped['TONS_IC'], label='آتش زا')
 plt.plot(dataGrouped.index, dataGrouped['TONS_HE'], label='انفجار قوی')
 plt.legend()
 plt.show()

        with bokeh      

In [142]:
p = figure(x_axis_type= 'datetime')


p.line(x= 'MSNDATE', y= 'TOTAL_TONS', source= dataSource, legend_label= 'کل انفجارها', line_width= 2)
p.line(x= 'MSNDATE', y= 'TONS_FRAG', color= 'blue', source= dataSource, legend_label= 'قطعات', line_width= 2)
p.line(x= 'MSNDATE', y= 'TONS_IC', color= 'green', source= dataSource, legend_label= 'آتش زا', line_width= 2)
p.line(x= 'MSNDATE', y= 'TONS_HE', color= 'red', source= dataSource, legend_label= 'انفجار قوی', line_width= 2)

# cm = factor_cmap(field_name= 'COUNTRY_FLYING_MISSION', palette= Spectral3, factors= countries)

# p.vbar_stack(source= dataSource, x= 'COUNTRY_FLYING_MISSION',
#                 width= 0.7, color= Spectral3,
#                 stackers= ['TONS_FRAG', 'TONS_IC', 'TONS_HE'],
#                 legend_label= ['قطعات', 'اشتعال زا', 'انفجار قوی']
# )

p.legend.visible = True

h = HoverTool()
h.tooltips = [
    ('جمع مواد منفجره تفکیک قوی', '@TONS_HE'),
    ('جمع مواد آتش زا', '@TONS_IC'),
    ('جمع تعداد قطعه منفجر شده', '@TONS_FRAG'),
    ('TOTAL_TONS', '@TOTAL_TONS')
]
h.mode= 'mouse'
p.add_tools(h)

p.legend.click_policy= 'hide'
show(p)